In [5]:
import cv2
import tensorflow as tf

In [2]:
img = cv2.imread('../INCOMING/2.jpg')
img.shape

(224, 224, 3)

In [6]:
generator = trainGenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 50,
    width_shift_range = 2.0,
    height_shift_range = 2.0,    
    shear_range = 0.4,
    zoom_range = 2.0,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

In [8]:
generator.zoom_range

[-1.0, 3.0]

In [12]:
transform = {
    'rescale' : 1./255,
    'rotation_range' : 50,
    'width_shift_range' : 2.0,
    'height_shift_range' : 2.0,
    'shear_range' : 0.4,
    'zoom_range' : 2.0,
    'horizontal_flip' : True,
    'vertical_flip' : True,
    'fill_mode' : 'nearest'
}

In [19]:
train = generator.flow_from_directory(
    'INCOMING\2.jpg',
    target_size = (225, 225),
    batch_size = 9,
    class_mode = 'binary'
)

NotADirectoryError: [WinError 267] The directory name is invalid: '../INCOMING/2.jpg'

In [18]:
generator.apply_transform(img, transform).shape

(224, 224, 3)